In [1]:
# !pip install sagemaker==1.38.6

In [2]:
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.tensorflow import TensorFlow

In [3]:
import boto3
import random
import json
import importlib
import bidding_data
import config

In [4]:
bidding_data.download_stats_file_from_s3()

In [5]:
# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'wsbidder'

# Location where results of model training are saved.
output_path = 's3://{}/trainer_predict_imp/model_output'.format(bucket)
model_dir = 's3://{}/trainer_predict_imp/model'.format(bucket)

# train_instance_type = 'ml.c4.xlarge'
train_instance_type = 'ml.p2.xlarge'
train_instance_count = 1
hyperparameters = {'epochs': 50000, 'batch_size': 512, 'config_file': 'config.json'}
inputs = {'train': 's3://wsbidder/trainer_predict_imp/data/train/',
          'eval': 's3://wsbidder/trainer_predict_imp/data/eval/',
          'test': 's3://wsbidder/trainer_predict_imp/data/eval/'}
# inputs = {'train': '/opt/ml/input/data/train/',
#           'eval': '/opt/ml/input/data/eval/',
#           'test': '/opt/ml/input/data/eval/'}
base_job_name = 'tf-imp-prediction'

In [6]:
estimator = TensorFlow(entry_point='train_predict_imp.py',
                       source_dir='.',
                       output_path=output_path,
                       model_dir=model_dir,
                       train_instance_type=train_instance_type,
                       train_instance_count=train_instance_count,
                       hyperparameters=hyperparameters,
                       role=get_execution_role(),
#                        image_name='520713654638.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-tensorflow-scriptmode:1.12.0-gpu-py3',
                       base_job_name=base_job_name,
                       framework_version='1.14.0',
                       py_version='py3',
#                        distributions={'parameter_server': {'enabled': True}},
                       script_mode=True)

In [7]:
# from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# # Define loss objective
# loss_objective_metric_name = 'loss'
# loss_objective_type = 'Minimize'
# loss_metric_definitions = [{'Name': 'loss',
#                        'Regex': 'loss = ([0-9\\.]+)'}]

# # acc_objective_metric_name = 'accuracy'
# # acc_objective_type = 'Maximize'
# # acc_metric_definitions = [{'Name': 'accuracy',
# #                        'Regex': 'accuracy = ([0-9\\.]+)'}]

# # Define hyperparameter ranges
# hyperparameter_ranges = {
#                             'learning_rate': ContinuousParameter(1e-6, 1e-4),
#                             'dropout_rate': ContinuousParameter(0.0, 0.5),
#                             'layer_number': IntegerParameter(4, 12),
#                             'node_number': IntegerParameter(128, 512)
#                         }  
# # hyperparameter_ranges = {
# #                             'learning_rate': ContinuousParameter(0.0000001, 0.0001),
# #                             'dropout_rate': ContinuousParameter(0.0, 0.9),
# #                             'batch_size': IntegerParameter(512, 4096)
# #                         }  
# # Initialise Sagemaker's hyperparametertuner
# tuner = HyperparameterTuner(estimator,
#                             loss_objective_metric_name,
#                             hyperparameter_ranges,
#                             loss_metric_definitions,
#                             max_jobs=16,
#                             max_parallel_jobs=4,
#                             objective_type=loss_objective_type)

In [8]:
s3 = boto3.resource('s3')
for obj in s3.Bucket(bucket).objects.filter(Prefix='trainer_predict_imp/model/'):
    s3.Object(bucket,obj.key).delete()

In [9]:
# tuner.fit(inputs)

In [10]:
# tuner.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='ImpressionPredictionProductionEndpoint',
#     update_endpoint=True
# )

In [1]:
# def update_model(model_data_url):
#     from sagemaker.tensorflow.serving import Model
#     from sagemaker import get_execution_role

#     model_data = model_data_url

#     model = Model(
#         model_data=model_data, 
#         role=get_execution_role(),
#         image='763104351884.dkr.ecr.eu-west-1.amazonaws.com/tensorflow-inference:1.14-cpu',
#         framework_version='1.14.0',
#         env= {
#             "SAGEMAKER_TFS_ENABLE_BATCHING":"true",
#             "SAGEMAKER_TFS_MAX_BATCH_SIZE": "32",
#             "SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS":"10000"
#           }
#     )
#     model.deploy(
#         initial_instance_count=1,
#         instance_type='ml.c5.large',
#         update_endpoint=True,
#         endpoint_name='ImpressionPredictionProductionEndpoint'
#     )
# model_data_url = 's3://wsbidder/trainer_predict_imp/model_output/tf-imp-prediction-2020-02-21-17-19-01-773/output/model.tar.gz'
# update_model(model_data_url)

-----------!

In [ ]:
%%time

estimator.fit(inputs)

2020-02-21 17:19:02 Starting - Starting the training job...
2020-02-21 17:19:03 Starting - Launching requested ML instances......
2020-02-21 17:20:29 Starting - Preparing the instances for training.........
2020-02-21 17:21:35 Downloading - Downloading input data......
2020-02-21 17:23:01 Training - Downloading the training image...
2020-02-21 17:23:22 Training - Training image download completed. Training in progress./usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uin

W0221 17:23:30.270904 140637592430336 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0221 17:23:30.475453 140637592430336 deprecation_wrapper.py:119] From train_predict_imp.py:186: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

I0221 17:23:30.476441 140637592430336 estimator.py:209] Using config: {'_model_dir': 's3://wsbidder/trainer_predict_imp/model/1582305808/', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, 

I0221 17:23:40.856951 140637592430336 session_manager.py:500] Running local_init_op.
I0221 17:23:40.928104 140637592430336 session_manager.py:502] Done running local_init_op.
I0221 17:23:44.166872 140637592430336 basic_session_run_hooks.py:606] Saving checkpoints for 0 into s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt.
I0221 17:23:48.668248 140637592430336 basic_session_run_hooks.py:262] loss = 0.88810575, step = 0
I0221 17:23:55.561838 140637592430336 basic_session_run_hooks.py:606] Saving checkpoints for 100 into s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt.
I0221 17:23:57.329614 140637592430336 estimator.py:1145] Calling model_fn.
I0221 17:23:57.329957 140637592430336 estimator.py:1145] Calling model_fn.
W0221 17:23:59.297267 140637592430336 deprecation.py:323] From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/metrics_impl.py:809: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instruc

I0221 17:24:41.847572 140637592430336 basic_session_run_hooks.py:606] Saving checkpoints for 300 into s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt.
I0221 17:24:43.617571 140637592430336 estimator.py:1145] Calling model_fn.
I0221 17:24:43.617890 140637592430336 estimator.py:1145] Calling model_fn.
W0221 17:24:45.976330 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0221 17:24:46.001945 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0221 17:24:46.027325 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 17:24:48.096611 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 17:24:48.121993 140637592430336 evaluation.py:255] Starting evaluation at 2020-02-21T17:24:48Z
I0221 17:24:48.544587 140637592430336 monitored_session.py:240] Graph was finalized.
I0221 17:

I0221 17:25:23.912436 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 17:25:23.912692 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 17:25:23.913324 140637592430336 export_utils.py:170] Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
I0221 17:25:23.913451 140637592430336 export_utils.py:170] Signatures INCLUDED in export for Regress: None
I0221 17:25:23.913615 140637592430336 export_utils.py:170] Signatures INCLUDED in export for Predict: ['predict']
I0221 17:25:23.913739 140637592430336 export_utils.py:170] Signatures INCLUDED in export for Train: None
I0221 17:25:23.913859 140637592430336 export_utils.py:170] Signatures INCLUDED in export for Eval: None
I0221 17:25:24.250266 140637592430336 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt-400
I0221 17:25:24.713543 140637592430336 builder_impl.py:661] Assets added to graph.
I0221 17:25:24.713788 140637592430336 buil

...

I0221 20:07:04.030164 140637592430336 evaluation.py:167] Evaluation [10/100]
I0221 20:07:04.512017 140637592430336 evaluation.py:167] Evaluation [20/100]
I0221 20:07:05.001849 140637592430336 evaluation.py:167] Evaluation [30/100]
I0221 20:07:05.480664 140637592430336 evaluation.py:167] Evaluation [40/100]
I0221 20:07:06.036160 140637592430336 evaluation.py:167] Evaluation [50/100]
I0221 20:07:06.532687 140637592430336 evaluation.py:167] Evaluation [60/100]
I0221 20:07:07.038355 140637592430336 evaluation.py:167] Evaluation [70/100]
I0221 20:07:07.524338 140637592430336 evaluation.py:167] Evaluation [80/100]
I0221 20:07:08.032860 140637592430336 evaluation.py:167] Evaluation [90/100]
I0221 20:07:08.527501 140637592430336 evaluation.py:167] Evaluation [100/100]
I0221 20:07:08.858891 140637592430336 evaluation.py:275] Finished evaluation at 2020-02-21-20:07:08
I0221 20:07:08.859289 140637592430336 estimator.py:2039] Saving dict for global step 45600: accuracy/imp_4 = 0.7811458, accuracy/

W0221 20:08:02.252723 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0221 20:08:02.282147 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0221 20:08:02.309385 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 20:08:04.425832 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 20:08:04.451143 140637592430336 evaluation.py:255] Starting evaluation at 2020-02-21T20:08:04Z
I0221 20:08:04.861392 140637592430336 monitored_session.py:240] Graph was finalized.
I0221 20:08:04.955754 140637592430336 saver.py:1286] Restoring parameters from s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt-45900
I0221 20:08:05.689299 140637592430336 session_manager.py:500] Running local_init_op.
I0221 20:08:05.839704 140637592430336 session_manager.py:502] Done running local_init_op.
I0

I0221 20:08:52.245697 140637592430336 evaluation.py:167] Evaluation [30/100]
I0221 20:08:52.827791 140637592430336 evaluation.py:167] Evaluation [40/100]
I0221 20:08:53.394433 140637592430336 evaluation.py:167] Evaluation [50/100]
I0221 20:08:53.974267 140637592430336 evaluation.py:167] Evaluation [60/100]
I0221 20:08:54.545135 140637592430336 evaluation.py:167] Evaluation [70/100]
I0221 20:08:55.126489 140637592430336 evaluation.py:167] Evaluation [80/100]
I0221 20:08:55.705824 140637592430336 evaluation.py:167] Evaluation [90/100]
I0221 20:08:56.283059 140637592430336 evaluation.py:167] Evaluation [100/100]
I0221 20:08:56.618660 140637592430336 evaluation.py:275] Finished evaluation at 2020-02-21-20:08:56
I0221 20:08:56.619065 140637592430336 estimator.py:2039] Saving dict for global step 46100: accuracy/imp_4 = 0.8351857, accuracy/imp_6 = 0.7372634, accuracy/imp_7 = 0.6509211, auc/predict_imp = 0.82538426, auc_PR/imp_4 = 0.45770854, auc_PR/imp_6 = 0.8072991, auc_PR/imp_7 = 0.6686517

I0221 20:09:44.318450 140637592430336 basic_session_run_hooks.py:606] Saving checkpoints for 46400 into s3://wsbidder/trainer_predict_imp/model/1582305808/model.ckpt.
I0221 20:09:46.598272 140637592430336 estimator.py:1145] Calling model_fn.
I0221 20:09:46.598600 140637592430336 estimator.py:1145] Calling model_fn.
W0221 20:09:48.942508 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W0221 20:09:48.968339 140637592430336 metrics_impl.py:804] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
I0221 20:09:48.994061 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 20:09:50.889270 140637592430336 estimator.py:1147] Done calling model_fn.
I0221 20:09:50.914470 140637592430336 evaluation.py:255] Starting evaluation at 2020-02-21T20:09:50Z
I0221 20:09:51.581252 140637592430336 monitored_session.py:240] Graph was finalized.
I0221 2

I0221 20:10:42.664285 140637592430336 evaluation.py:167] Evaluation [100/100]
I0221 20:10:43.024181 140637592430336 evaluation.py:275] Finished evaluation at 2020-02-21-20:10:43
I0221 20:10:43.024597 140637592430336 estimator.py:2039] Saving dict for global step 46600: accuracy/imp_4 = 0.8372871, accuracy/imp_6 = 0.72609705, accuracy/imp_7 = 0.65185654, auc/predict_imp = 0.8357621, auc_PR/imp_4 = 0.4611951, auc_PR/imp_6 = 0.79151106, auc_PR/imp_7 = 0.66989744, auc_precision_recall/predict_imp = 0.8263089, auc_roc/imp_4 = 0.70477694, auc_roc/imp_6 = 0.5172532, auc_roc/imp_7 = 0.65346575, average_loss/predict_imp = 0.49313772, global_step = 46600, loss = 0.7374109, precision/imp_4 = 0.63559955, precision/imp_6 = 0.79191583, precision/imp_7 = 0.70359075, recall/imp_4 = 0.48043546, recall/imp_6 = 0.88329554, recall/imp_7 = 0.6379647
I0221 20:10:43.186862 140637592430336 estimator.py:2099] Saving 'checkpoint_path' summary for global step 46600: s3://wsbidder/trainer_predict_imp/model/158230

In [ ]:
# estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.c5.large',
#     endpoint_name='ImpressionPredictionProductionEndpoint',
#     update_endpoint=True
# )

In [ ]:
# estimator.delete_endpoint()